In [ ]:
# Cargamos las librerias necesarias
from pathlib import Path
import pandas as pd
import unicodedata



In [7]:
# Directorio donde está el notebook
BASE_DIR = Path.cwd()
directorio = BASE_DIR / "Balance_DIARIO_Raw"

extension=".xlsx"
nombres_archivos = ["balance-electrico_01-01-2019_31-12-2020", "balance-electrico_01-01-2021_31-12-2022", "balance-electrico_01-01-2023_31-12-2024"]


In [ ]:
#_____________________________________________________________________________________
# 03.BALANCE Eléctrico: FUNCIÓN ESPECÍFICA PARA ARCHIVO descargado desde OMIE - Datos Diarios
#_____________________________________________________________________________________

# FUNCIÓN B03_2: Generar DataFrame total desde los parciales y guardar CSV

def generate_df_balance_total (directorio, BASE_DIR, nombres_archivos, extension):
    
    df_balance_total = pd.DataFrame() # Inicializar df_total
    for archivo in nombres_archivos:
        df_balance_total = pd.concat([df_balance_total, generate_df_balance_partial(directorio, archivo, extension)], ignore_index=True)


    df_balance_total['DayOfYear'] = pd.to_datetime({'year': df_balance_total['Year'], 'month': df_balance_total['Month'], 'day': df_balance_total['Day']}).dt.dayofyear
        
      
      # Imputamos en aquellos casos donde sea Null la demanda
    df_balance_total.loc[df_balance_total['Demanda'].isna(), 'Demanda'] = (
    df_balance_total.loc[df_balance_total['Demanda'].isna(), ['Generacion renovable', 'Generacion no renovable', 'Saldo almacenamiento', 'Enlace Peninsula-Baleares','Saldo I. internacionales']].sum(axis=1)
)
      
    # Crear el nuevo DataFrame con las columnas que te interesan
    df_balance_red = df_balance_total[[
        'Year', 'Month', 'Day', 'DayOfYear',
        'Demanda', 'Eolica', 
        'Solar fotovoltaica', 'Solar termica'
    ]].copy()  # copia para evitar advertencias de asignación

    # Crear la nueva columna 'Solar' como suma de las dos columnas solares
    df_balance_red['Solar'] = (
        df_balance_red['Solar fotovoltaica'] + df_balance_red['Solar termica']
    )
    # Eliminamos las dos columnas originales:
    df_balance_red = df_balance_red.drop(columns=['Solar fotovoltaica', 'Solar termica'])
        
    # Guardar CSV limpio 
    
    
    df_balance_total.to_csv(
    f"{BASE_DIR}/balance_TOTAL.csv",
    sep=";",          # separador de columnas
    index=False,
    encoding="utf-8",
    decimal=",",      # usa coma como separador decimal en el archivo
    float_format="%.6f"  # controla cuántos decimales (ajusta a tu necesidad)
)

    df_balance_red.to_csv(
        f"{BASE_DIR}/balance_REDUCIDO.csv",
        sep=";",          # separador de columnas
        index=False,
        encoding="utf-8",
        decimal=",",      # usa coma como separador decimal en el archivo
        float_format="%.6f"  # controla cuántos decimales (ajusta a tu necesidad)
    )
    
    return df_balance_total, df_balance_red

# FUNCIÓN B03_1: Generar DataFrame parcial desde xlsx
    
def generate_df_balance_partial(directorio, nombre_archivo, extension): 
    ruta = directorio / f"{nombre_archivo}{extension}"
    df = pd.read_excel(ruta)
    #df = pd.read_excel(ruta, converters=lambda x: str(x).replace(",", "."))
    ultima_columna = df.columns[(df.notna().any())][-1]
        # Leer el rango específico de celdas (A1:ABD33)
        # Ajustar según la estructura del archivo
        # La primera fila (A1) es la cabecera, las filas 2 a 4 son metadatos y se saltan
        # Se leen 29 filas (de la 5 a la 33 inclusive) y las columnas desde A hasta ABD
        # La primera columna (A) se usa como índice
        # df = pd.read_excel(ruta, header=0, skiprows=4, nrows=29, usecols="A:ABD", index_col=0)
        
        # Leer el rango específico de celdas (A1:ABD33) usando usecols con range
        # Esto es útil si no se conoce la letra de la última columna o si puede variar
        # Aquí se asume que la última columna con datos es ABD (columna 56, índice 55)
        # Ajustar ultima_columna si es necesario
    ultima_columna_index = df.columns.get_loc(ultima_columna) + 1  # +1 porque range es exclusivo en el extremo superior
        
        # Leer el archivo Excel con los parámetros especificados    
    df = pd.read_excel(
        ruta,
        header=0,            # La primera fila del rango será la cabecera
        skiprows=4,          # Salta las primeras 4 filas (A1:A4)
        nrows=29,            # Lee 29 filas (de la 5 a la 33 inclusive)
        usecols=range(0, ultima_columna_index),     # Columnas desde A hasta ABD
        index_col=0          # La primera columna como índice
    )

   
    df=df.transpose().reset_index()
    # tRANSPONER EL DATAFRAME

    # Convertir a float todas las columnas menos la primera
    df.replace("-", 0, inplace=True)
    df.iloc[:, 1:] = df.iloc[:, 1:].astype('float64')
        
        # Crear nuevas columnas
    df['Year'] = df['index'].str[7:9].astype(int)+2000
        
    meses_dict = {
        'ene': 1, 'feb': 2, 'mar': 3, 'abr': 4, 'may': 5, 'jun': 6,
        'jul': 7, 'ago': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dic': 12
    }
    df['Month'] = df['index'].str[3:6].map(meses_dict)
    df['Day'] = df['index'].str[0:2].astype(int)
    
    df = df.rename(columns={'Demanda en b.c.': 'Demanda'})

    # Eliminamos las dos columnas que no aportan
    df.drop(columns=['index'], inplace=True)
    # df.drop(df.columns[-4], axis=1, inplace=True)

    # Mover las tres últimas columnas al principio
    cols = list(df.columns)
    new_order = cols[-3:] + [cols[-4], cols[-5]] + cols[:-5]
    df = df[new_order]
    
    # Normalizar nombres de columnas para eliminar acentos
    df.columns = [unicodedata.normalize("NFKD", c).encode("ascii", "ignore").decode("utf-8") for c in df.columns]
    return df 

In [9]:
# Lee el rango A5:ABD33 (A=1, ABD=730, fila 5 a 33)
df_balance_total, df_balance_red = generate_df_balance_total (directorio, BASE_DIR, nombres_archivos, extension)


C:\Users\Elena\AppData\Local\Temp\ipykernel_648\3520147123.py:95: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("-", 0, inplace=True)
C:\Users\Elena\AppData\Local\Temp\ipykernel_648\3520147123.py:95: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("-", 0, inplace=True)
C:\Users\Elena\AppData\Local\Temp\ipykernel_648\3520147123.py:95: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False

In [10]:
def explora_df(df,title_1=""):
    print(f"------------EJEMPLO CABECERAS-{title_1}-------------\n--------------------------")
    print(df.head())
    print(f"------------INFO TIPO DATOS-{title_1}---------------\n--------------------------")
    df.info()
    print(f"------------ESTADÍSTICAS-{title_1} -----------------\n--------------------------")
    print(df.describe(include='all').transpose())
    
explora_df(df_balance_red)

------------EJEMPLO CABECERAS--------------
--------------------------
   Year  Month  Day  DayOfYear     Demanda      Eolica      Solar
0  2019      1    1          1  547.404192   77.680013  25.586870
1  2019      1    2          2  701.784804  194.507772  25.453782
2  2019      1    3          3  746.345169  109.405705  22.307852
3  2019      1    4          4  754.085459   79.098774  23.624898
4  2019      1    5          5  681.461272  116.074127  25.952103
------------INFO TIPO DATOS----------------
--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192 entries, 0 to 2191
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       2192 non-null   int64  
 1   Month      2192 non-null   int64  
 2   Day        2192 non-null   int64  
 3   DayOfYear  2192 non-null   int32  
 4   Demanda    2192 non-null   float64
 5   Eolica     2192 non-null   float64
 6   Solar      2192 non-null   floa